In [ ]:
#This was a Master of Management Analytics assignment so I can't link the exact dataset, but a near identical one can be found here: https://www.kaggle.com/code/benroshan/sentiment-analysis-amazon-reviews/data


# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.books_5_small")
df3 = spark.sql("select * from default.home_and_kitchen_5_small")
df = df1.union(df2).union(df3)
df = df.na.drop(subset=["reviewText", "label"])
# df= df.sample(False, 0.15, seed=47)
# df = df.dropna()
# rebalance dataset via downsampling
df_helpful = df.filter(df.label == 1)
df_unhelpful = df.filter(df.label == 0)

imbalance = df_helpful.count() / df_unhelpful.count()

df_unhelpful = df_unhelpful.sample(False, imbalance, seed=47)
df = df_unhelpful.union(df_helpful)

print((df.count(), len(df.columns)))

(1252736, 11)

In [ ]:
df.printSchema()

In [ ]:
import datetime
from pyspark.sql.functions import from_utc_timestamp, from_unixtime, year, month, dayofmonth
from pyspark.sql.types import StringType

df = df.withColumn("date", from_unixtime(df['unixReviewTime']).cast(StringType()))

df = df.withColumn("year", year(df["date"].cast(StringType())))
df = df.withColumn("month", month(df["date"].cast(StringType())))

In [ ]:
from transformers import pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from pyspark.sql.functions import StringType
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as func

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer


In [ ]:
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity
def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, DoubleType())
    words = words.withColumn("polarity", polarity_detection_udf("reviewText"))
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, DoubleType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("reviewText"))
    return words


In [ ]:
def data_preprocessing(df_orig):
  """
  UDF to handle the E2E pre-processing before model training
  """
  to_drop = ['reviewID', 'reviewerName','reviewTime','summary']
  # Condition to apply pre-processing to Kaggle test data
  if 'label' not in df_orig.columns:
    to_drop.remove('reviewID')
    
  df = df_orig.select([column for column in df_orig.columns if column not in to_drop])
  
  ## Add in text polarity & subjectivity  
  
  ## Retrieve review years and months
  df = df.withColumn('year', year(from_unixtime(col('unixReviewTime'))))
  df = df.withColumn('month', month(from_unixtime(col('unixReviewTime'))))

  ####### Begin new FE attempt iterations ################
  initial_df_columns = df.columns
  
  # Compute number of reviews by Reviewer
  to_join = df.groupBy('reviewerID').count()
  df = df.join(to_join, ['reviewerID'], 'left').select(col('count').alias('reviewerNumReviews'),*initial_df_columns).drop('reviewerID')
  initial_df_columns.remove('reviewerID')
  #df = df.withColumn
  #df = df.groupBy('reviewerID').count().select(col('count').alias('reviewerNumReviews'), *initial_df_columns)
  
  # Compute time since first product review and rating deviation from product average
  to_join = df.groupBy('asin').agg(min('unixReviewTime').alias('firstProductReview'),mean('overall').alias('meanScore'))
  df = df.join(to_join, ['asin'], 'left').select('firstProductReview', 'meanScore', 'reviewerNumReviews', *initial_df_columns)
  df = df.withColumn('sinceFirstReview', round(months_between(to_date(from_unixtime(col('unixReviewTime'))),to_date(from_unixtime(col('firstProductReview')))),2))\
  .withColumn('meanScoreDev', col('overall') - col('meanScore')).drop('firstProductReview','meanScore','asin','unixReviewTime')
  
  # Compute 
  #to_join = df.groupBy('asin').agg(mean('overall').alias('meanScore'))
  #df = df.join(to_join, ['asin'], 'left').select('meanScore', *initial_df_columns)
  #df = df.withColumn('meanScoreDev', col('overall') - col('meanScore')).drop('meanScore','asin')
  
  return df

In [ ]:
df = data_preprocessing(df)

In [ ]:
# set seed for reproducibility
# (trainingData, testingData) = df.randomSplit([0.8, 0.2], seed = 45)
(trainingData, testingData) = df.randomSplit([0.8, 0.2], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testingData.count()))

Training Dataset Count: 1001180
Test Dataset Count: 251556

In [ ]:
# Data Preprocessing

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, OneHotEncoder, VectorAssembler

# We'll tokenize the text using a simple RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")


# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")


# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors
countVectors = CountVectorizer(inputCol="filtered", outputCol="countVector_output", vocabSize=10000, minDF=5)

# one hot encoding for year and month
year_encoder = OneHotEncoder(inputCol='year', outputCol='yearVec')
month_encoder = OneHotEncoder(inputCol='month', outputCol='monthVec')


# vector assembler
# assembler = VectorAssembler(inputCols=['countVector_output', 'yearVec', 'monthVec', 'polarity', 'subjectivity'],
#                             outputCol='features')

assembler = VectorAssembler(inputCols=['countVector_output', 'yearVec', 'monthVec', 'sinceFirstReview'],
                            outputCol='features')


pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, year_encoder, month_encoder, assembler])


In [ ]:
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(trainingData)
trainingDataTransformed = pipelineFit.transform(trainingData)
trainingDataTransformed.show(5)

+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+
reviewerNumReviews|overall|verified| reviewText|label| date|year|month|sinceFirstReview| meanScoreDev| words| filtered| countVector_output| yearVec| monthVec| features|
+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+
 1| 1.0| false|"Soul" turns out ...| 1|2003-12-11 00:00:00|2003| 12| 59.97|-3.1311475409836067|[soul, turns, out...|[soul, turns, som...|(10000,[0,1,3,10,...|(2018,[2003],[1.0])| (12,[],[])|(12032,[0,1,3,10,...|
 1| 1.0| false|<div id="video-bl...| 0|2016-03-17 00:00:00|2016| 3| 40.74| -3.0|[div, id, video, ...|[div, id, video, ...|(10000,[0,3,4,27,...|(2018,[2016],[1.0])|(12,[3],[1.0])|(12032,[0,3,4,27,...|
 1| 1.0| false|A long long long ...| 0|2016-02-07 00:00:00|2016| 2| 13.26|-2.8085106382978724|[a, long, long, l...|[long, long, long...|(10000,[4,5,19,21...|(2018,[2016],[1.0])|(12,[2],[1.0])|(12032,[4,5,19,21...|
 1| 1.0| false|ACK! This is the ...| 1|2009-01-24 00:00:00|2009| 1| 122.77| -3.0|[ack, this, is, t...|[ack, worst, book...|(10000,[1,3,6,8,9...|(2018,[2009],[1.0])|(12,[1],[1.0])|(12032,[1,3,6,8,9...|
 1| 1.0| false|After reading ove...| 1|2000-05-31 00:00:00|2000| 5| 23.42|-2.5555555555555554|[after, reading, ...|[reading, half, b...|(10000,[1,8,11,29...|(2018,[2000],[1.0])|(12,[5],[1.0])|(12032,[1,8,11,29...|
+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+
only showing top 5 rows

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingDataTransformed)

In [ ]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary


print("Training Accuracy:  " + str(trainingSummary.accuracy))
print("Training Precision: " + str(trainingSummary.precisionByLabel))
print("Training Recall:    " + str(trainingSummary.recallByLabel))
print("Training FMeasure:  " + str(trainingSummary.fMeasureByLabel()))
print("Training AUC:       " + str(trainingSummary.areaUnderROC))

Training Accuracy: 0.778776044267764
Training Precision: [0.7411096903511197, 0.8303744733229181]
Training Recall: [0.8568389088112196, 0.7007240948813983]
Training FMeasure: [0.7947835352435663, 0.7600600161415254]
Training AUC: 0.8659888785091495

In [ ]:
testingDataTransform = pipelineFit.transform(testingData)
# testingDataTransform.show(5)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


predictions = lrModel.transform(testingDataTransform)
predictions.show(5)

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print('Test Area Under ROC', evaluator.evaluate(predictions))

+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+----------+
reviewerNumReviews|overall|verified| reviewText|label| date|year|month|sinceFirstReview| meanScoreDev| words| filtered| countVector_output| yearVec| monthVec| features| rawPrediction| probability|prediction|
+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+----------+
 1| 1.0| false|A friend very hig...| 1|2017-03-23 00:00:00|2017| 3| 205.19|-3.1386138613861387|[a, friend, very,...|[friend, highly, ...|(10000,[0,1,3,6,7...|(2018,[2017],[1.0])|(12,[3],[1.0])|(12032,[0,1,3,6,7...|[-3.5143941764073...|[0.02890543603046...| 1.0|
 1| 1.0| false|About 25 pages in...| 1|2011-01-05 00:00:00|2011| 1| 150.74| -3.092507645259939|[about, 25, pages...|[25, pages, book,...|(10000,[1,15,76,8...|(2018,[2011],[1.0])|(12,[1],[1.0])|(12032,[1,15,76,8...|[0.00209034397477...|[0.50052258580340...| 0.0|
 1| 1.0| false|Aggressively unhe...| 1|2007-08-26 00:00:00|2007| 8| 92.52| -1.875|[aggressively, un...|[aggressively, un...|(10000,[1,205,251...|(2018,[2007],[1.0])|(12,[8],[1.0])|(12032,[1,205,251...|[-0.6512842695364...|[0.34270018850667...| 1.0|
 1| 1.0| false|Although I realiz...| 1|2007-06-25 00:00:00|2007| 6| 44.26|-3.1764705882352944|[although, i, rea...|[although, realiz...|(10000,[0,1,3,7,1...|(2018,[2007],[1.0])|(12,[6],[1.0])|(12032,[0,1,3,7,1...|[-3.6626303594682...|[0.02502271015682...| 1.0|
 1| 1.0| false|As I bought this ...| 1|2009-03-22 00:00:00|2009| 3| 105.26|-3.2750000000000004|[as, i, bought, t...|[bought, book, ba...|(10000,[0,1,3,6,7...|(2018,[2009],[1.0])|(12,[3],[1.0])|(12032,[0,1,3,6,7...|[-5.9754234613167...|[0.00253398867585...| 1.0|
+------------------+-------+--------+--------------------+-----+-------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

Test Area Under ROC 0.8638159226410215

In [ ]:
# Load in the tables
holdout_df = spark.sql("select * from default.reviews_holdout")
holdout_df.show(5)
print((holdout_df.count(), len(holdout_df.columns)))

+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
67000000| 3.0| true|05 16, 2012|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading| 1337126400|
67000001| 5.0| true|12 25, 2012|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................| 1356393600|
67000002| 5.0| true|09 18, 2012|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...| 1347926400|
67000003| 5.0| false|07 27, 2015|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...| 1437955200|
67000004| 5.0| false|03 29, 2003| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...| 1048896000|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
only showing top 5 rows

(304984, 10)

In [ ]:
#holdout_preprocessing
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
# holdout_df = holdout_df.withColumn("date", from_unixtime(holdout_df['unixReviewTime']).cast(StringType()))
# holdout_df = holdout_df.withColumn("year", year(holdout_df["date"].cast(StringType())))
# holdout_df = holdout_df.withColumn("month", month(holdout_df["date"].cast(StringType())))
holdout_df = data_preprocessing(holdout_df)

# holdout_df = holdout_df.withColumn("overall", f.col('overall')/5)
# holdout_df = holdout_df.withColumn('verified', when(holdout_df.verified == "true", 1).otherwise(0))
# holdout_df = text_classification(holdout_df)

In [ ]:
holdout_df.show(5)

+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+
reviewerNumReviews|reviewID|overall|verified| reviewText|year|month|sinceFirstReview| meanScoreDev|
+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+
 3|67000164| 4.0| true|Story Rating ~ 4 ...|2015| 12| 0.35|-0.7999999999999998|
 1|67000321| 3.0| true|Well written - un...|2015| 9| 0.9|-0.7999999999999998|
 2|67000350| 5.0| false|This is an awesom...|2016| 6| 0.0| 0.0|
 1|67000615| 5.0| true|I. Love. This. Bo...|2016| 11| 0.0|0.33333333333333304|
 1|67000711| 5.0| true|Very good book fo...|2016| 3| 2.71| 0.75|
+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+
only showing top 5 rows

In [ ]:
holdout_df_Transform = pipelineFit.transform(holdout_df)
holdout_df_Transform.show(5)

+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+---------------+--------------------+
reviewerNumReviews|reviewID|overall|verified| reviewText|year|month|sinceFirstReview| meanScoreDev| words| filtered| countVector_output| yearVec| monthVec| features|
+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+---------------+--------------------+
 3|67000164| 4.0| true|Story Rating ~ 4 ...|2015| 12| 0.35|-0.7999999999999998|[story, rating, 4...|[story, rating, 4...|(10000,[0,1,10,14...|(2018,[2015],[1.0])| (12,[],[])|(12032,[0,1,10,14...|
 1|67000321| 3.0| true|Well written - un...|2015| 9| 0.9|-0.7999999999999998|[well, written, u...|[well, written, e...|(10000,[0,1,5,6,1...|(2018,[2015],[1.0])| (12,[9],[1.0])|(12032,[0,1,5,6,1...|
 2|67000350| 5.0| false|This is an awesom...|2016| 6| 0.0| 0.0|[this, is, an, aw...|[awesome, long, s...|(10000,[5,7,12,17...|(2018,[2016],[1.0])| (12,[6],[1.0])|(12032,[5,7,12,17...|
 1|67000615| 5.0| true|I. Love. This. Bo...|2016| 11| 0.0|0.33333333333333304|[i, love, this, b...|[love, book, revi...|(10000,[0,1,3,6,7...|(2018,[2016],[1.0])|(12,[11],[1.0])|(12032,[0,1,3,6,7...|
 1|67000711| 5.0| true|Very good book fo...|2016| 3| 2.71| 0.75|[very, good, book...|[good, book, begi...|(10000,[1,5,9,227...|(2018,[2016],[1.0])| (12,[3],[1.0])|(12032,[1,5,9,227...|
+------------------+--------+-------+--------+--------------------+----+-----+----------------+-------------------+--------------------+--------------------+--------------------+-------------------+---------------+--------------------+
only showing top 5 rows

In [ ]:
predictions = lrModel.transform(holdout_df_Transform)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

probelement=udf(lambda v:float(v[1]),FloatType())
submission_data = predictions.select('reviewID', probelement('probability')).withColumnRenamed('<lambda>(probability)', 'label')

In [ ]:
display(submission_data.select('reviewID', 'label'))

reviewID,label
67000164,0.73149127
67000321,0.90904415
67000350,0.6993052
67000615,0.69776225
67000711,0.36432266
67001316,0.27518848
67001360,0.26817036
67001606,0.34211093
67001796,0.8867972
67002455,0.4876544


# Build Gradient Boosting Model (suprisingly performed worse; hence commented out)

In [ ]:
# from pyspark.ml.classification import GBTClassifier
# ## Fitting the model
# gbt = GBTClassifier(maxIter=10)
# gbtModel = gbt.fit(trainingDataTransformed)



In [ ]:
# gbtPreds = gbtModel.transform(testingDataTransform)


In [ ]:
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# ## Evaluating the model
# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# gb_accuracy = evaluator.evaluate(gbtPreds)
# print("Accuracy of GBT is = %g"% (gb_accuracy))


In [ ]:
# evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
# print('Test Area Under ROC', evaluator.evaluate(gbtPreds))

In [ ]:
# gbt_test_df_Transform = pipelineFit.transform(test_df)

In [ ]:
# gbt_predictions = gbtModel.transform(test_df_Transform)

In [ ]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import FloatType

# probelement=udf(lambda v:float(v[1]),FloatType())
# submission_data = gbt_predictions.select('reviewID', probelement('probability')).withColumnRenamed('<lambda>(probability)', 'label')

In [ ]:
# display(submission_data.select('reviewID', 'label'))